First, use PostgreSQL's dropdb command to drop the database named proj2, if it exists.

## Part 3

In [1]:
!dropdb -U student proj2

Now use PostgreSQL's createdb command to create the database named proj2.

In [2]:
!createdb -U student proj2

In [3]:
%load_ext sql

In [4]:
%sql postgresql://student@/proj2

'Connected: student@proj2'

In [5]:
!wget -O file.csv https://s3.amazonaws.com/dmfa-2017/bike_stations.csv

--2018-10-12 18:13:06--  https://s3.amazonaws.com/dmfa-2017/bike_stations.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.82.99
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.82.99|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 169879 (166K) [text/csv]
Saving to: ‘file.csv’

file.csv            100%[===================>] 165.90K  --.-KB/s    in 0.004s  

2018-10-12 18:13:07 (40.9 MB/s) - ‘file.csv’ saved [169879/169879]



First, we take a look at a sample of the data to determine its attributes' domains and ranges.

Based on these values, we expect we can work with the following:

__1.Members__

In [6]:
%%sql

DROP TABLE IF EXISTS Members;

CREATE TABLE Members (
    SSN      VARCHAR(20) NOT NULL PRIMARY KEY,
    Name     VARCHAR(60) NOT NULL, 
    Address  VARCHAR(60) NOT NULL
); 

 * postgresql://student@/proj2
Done.
Done.


[]

In [7]:
%%sql

DROP TABLE IF EXISTS Memberships;

CREATE TABLE Memberships (
    Key_ID      INTEGER NOT NULL PRIMARY KEY,
    SSN         VARCHAR(20) NOT NULL REFERENCES Members,
    Type        VARCHAR(20) CHECK (Type in ('annual','semiannual','monthly')),  
    Status      CHAR(10) CHECK (status in ('active','inactive') ) ,
    Start_Date  Date NOT NULL,
    End_Date    Date NOT NULL
); 

 * postgresql://student@/proj2
Done.
Done.


[]

In [8]:
%%sql

DROP TABLE IF EXISTS Stations;

CREATE TABLE Stations (
    Station_ID     INTEGER NOT NULL PRIMARY KEY,
    Station_Name   CHAR(100) NOT NULL,
    Capacity       VARCHAR(4)
); 

 * postgresql://student@/proj2
Done.
Done.


[]

In [9]:
%%sql

DROP TABLE IF EXISTS Bikes;

CREATE TABLE Bikes (
    Bike_ID                CHAR(10) NOT NULL PRIMARY KEY,
    Availability           VARCHAR(20) CHECK (availability in ('serviceable','disable','check out') ) ,
    Dorked_Station_ID      INTEGER REFERENCES Stations(Station_ID)
); 

 * postgresql://student@/proj2
Done.
Done.


[]

In [10]:
%%sql
DROP TABLE IF EXISTS Rides;

CREATE TABLE Rides (
    Ride_ID             INTEGER NOT NULL PRIMARY KEY,
    Bike_ID             CHAR(10) NOT NULL ,
    Start_Station_ID    INTEGER NOT NULL REFERENCES Stations(Station_ID),
    Key_ID              INTEGER NOT NULL REFERENCES Memberships(Key_ID),
    Start_Time          Timestamp NOT NULL
); 

 * postgresql://student@/proj2
Done.
Done.


[]

### Fill the tables with data

In [11]:
pwd!

'/home/ubuntu/Class 5/Project 2'

__Temporary Table__

In [12]:
%%sql
DROP TABLE IF EXISTS temp;

CREATE TABLE temp (
    bike_id VARCHAR(21),
    status VARCHAR(1),
    station_id INTEGER,
    station_name VARCHAR(64),
    capacity INTEGER
);

 * postgresql://student@/proj2
Done.
Done.


[]

In [13]:
%%sql
COPY temp FROM '/home/ubuntu/Class 5/Project 2/file.csv'
CSV
HEADER;

 * postgresql://student@/proj2
4317 rows affected.


[]

__Stations__

In [14]:
%%sql
COPY Stations FROM '/home/ubuntu/Class 5/Project 2/station.csv'
CSV
HEADER;

 * postgresql://student@/proj2
452 rows affected.


[]

In [15]:
%%sql
SELECT * 
FROM Stations
LIMIT 5;

 * postgresql://student@/proj2
5 rows affected.


station_id,station_name,capacity
32011,Maple & Ritchie Ave,28
31246,M St & Pennsylvania Ave NW,10
31408,Takoma Metro,12
31309,Fessenden St & Wisconsin Ave NW,8
31125,15th & W St NW,24


__Members__

In [16]:
%%sql
COPY Members FROM '/home/ubuntu/Class 5/Project 2/Members.csv'
CSV
HEADER;

 * postgresql://student@/proj2
10 rows affected.


[]

In [17]:
%%sql
SELECT * 
FROM Members;

 * postgresql://student@/proj2
10 rows affected.


ssn,name,address
579052106,Annmarie Aaron,4233 Fairfax Dr
577228101,Brandy Bennett,1100 Wilson Blvd
577428570,Cameron Cooper,301 Water St SE
579842899,Daina Devine,1300 Constitution Ave NW
579721097,Earleen Ernst,4212 East Capitol St NE
577928610,Fernanda Forrest,3700 O St NW
577039958,Jane Lee,0 MORGAN AVE
578926274,David Potter,120 BROAD MEADOW RD
578788989,Lily Zhang,PO BOX 217
577371597,Jack Jones,196 LINCOLN ST


__Memberships__

In [18]:
%%sql
COPY Memberships FROM '/home/ubuntu/Class 5/Project 2/memberships.csv'
CSV
HEADER;

 * postgresql://student@/proj2
10 rows affected.


[]

In [19]:
%%sql
SELECT * 
FROM Memberships;

 * postgresql://student@/proj2
10 rows affected.


key_id,ssn,type,status,start_date,end_date
1,579052106,annual,inactive,2016-10-01,2017-10-01
2,577228101,semiannual,inactive,2017-04-02,2017-10-02
3,577428570,monthly,inactive,2017-06-03,2017-07-03
4,579842899,semiannual,inactive,2017-06-04,2017-12-04
5,579721097,semiannual,active,2018-09-05,2019-03-05
6,577928610,monthly,active,2018-10-06,2018-11-06
7,577039958,annual,active,2018-10-07,2019-10-07
8,578926274,monthly,active,2018-10-07,2018-11-07
9,578788989,semiannual,active,2018-10-09,2019-04-09
10,577371597,annual,active,2017-10-10,2018-10-10


In [20]:
%%sql
COPY Bikes FROM '/home/ubuntu/Class 5/Project 2/bike.csv'
CSV
HEADER;

 * postgresql://student@/proj2
4298 rows affected.


[]

In [22]:
%%sql
SELECT * 
FROM Bikes
LIMIT 5;

 * postgresql://student@/proj2
5 rows affected.


bike_id,availability,dorked_station_id
W00001,serviceable,31301
W00005,serviceable,32011
W00007,serviceable,31508
W00009,serviceable,31912
W00010,serviceable,31249
